In [3]:
import smaract.ctl as ctl
import numpy as np
import time

In [4]:
locator = ctl.FindDevices()
d_handle = ctl.Open(locator)

In [5]:
locator

'usb:sn:MCS2-00003775'

### Get properties

In [6]:
for p in ctl.Property:
    print(p)

Property.NUMBER_OF_CHANNELS
Property.NUMBER_OF_BUS_MODULES
Property.INTERFACE_TYPE
Property.DEVICE_STATE
Property.DEVICE_SERIAL_NUMBER
Property.DEVICE_NAME
Property.EMERGENCY_STOP_MODE
Property.DEFAULT_EMERGENCY_STOP_MODE
Property.NETWORK_DISCOVER_MODE
Property.NETWORK_DHCP_TIMEOUT
Property.POWER_SUPPLY_ENABLED
Property.NUMBER_OF_BUS_MODULE_CHANNELS
Property.MODULE_TYPE
Property.MODULE_STATE
Property.STARTUP_OPTIONS
Property.AMPLIFIER_ENABLED
Property.AMPLIFIER_MODE
Property.POSITIONER_CONTROL_OPTIONS
Property.ACTUATOR_MODE
Property.CONTROL_LOOP_INPUT
Property.SENSOR_INPUT_SELECT
Property.POSITIONER_TYPE
Property.POSITIONER_TYPE_NAME
Property.MOVE_MODE
Property.CHANNEL_TYPE
Property.CHANNEL_STATE
Property.POSITION
Property.TARGET_POSITION
Property.SCAN_POSITION
Property.SCAN_VELOCITY
Property.HOLD_TIME
Property.MOVE_VELOCITY
Property.MOVE_ACCELERATION
Property.MAX_CL_FREQUENCY
Property.DEFAULT_MAX_CL_FREQUENCY
Property.STEP_FREQUENCY
Property.STEP_AMPLITUDE
Property.FOLLOWING_ERROR_LIM

In [7]:
n_channels_tot = ctl.GetProperty_i32(
            d_handle, 0, ctl.Property.NUMBER_OF_CHANNELS)
n_channels = np.sum([ctl.GetProperty_i32(d_handle, channel, ctl.Property.CHANNEL_STATE) & ctl.ChannelState.SENSOR_PRESENT != 0 for channel in range(n_channels_tot)])
print('N channels: ', n_channels)
base_units = [ctl.GetProperty_i32(
            d_handle, channel, ctl.Property.POS_BASE_UNIT) for channel in range(n_channels)]
print('Base units: ', ['pm' if bu==ctl.BaseUnit.METER else 'deg' for bu in base_units])#
positions = [ctl.GetProperty_i64(d_handle, channel, ctl.Property.POSITION)/1000 for channel in range(4)]
print('Current position: ', positions)

N channels:  4
Base units:  ['pm', 'pm', 'pm', 'deg']
Current position:  [-538671.753, 317874.721, 2937138.619, -14998813.971]


In [17]:
ch_type = [ctl.GetProperty_s(
    d_handle, i, ctl.Property.POSITIONER_TYPE_NAME) for i in range(n_channels)]
print('Channel types: ', ch_type)
cl_freq = [ctl.GetProperty_i32(
    d_handle, i, ctl.Property.MAX_CL_FREQUENCY) / 1000 for i in range(n_channels)]
print('Frequency [kHz]: ', cl_freq)

Channel types:  ['SL...D1SS', 'SL...D1SS', 'SL...D1SS', 'SR...S1S6S']
Frequency [kHz]:  [7.0, 7.0, 7.0, 14.0]


In [18]:
for i in range(n_channels):
    v = ctl.GetProperty_i64(d_handle, i, ctl.Property.MOVE_VELOCITY)
    if base_units[i] == ctl.BaseUnit.METER:
        print('Velocity: {}um/s'.format(v/10**6))
    else:
        print('Velocity: {}deg/s'.format(v/10**9))

Velocity: 0.0um/s
Velocity: 0.0um/s
Velocity: 0.0um/s
Velocity: 0.0deg/s


### Set the Max CL frequency and speed

In [19]:
import smaract.ctl as ctl
locator = ctl.FindDevices()
# d_handle = ctl.Open(locator)
for channel in range(4):
    if channel < 3:
        # linear positioners
        ctl.SetProperty_i32(
                    d_handle, channel, ctl.Property.DEFAULT_MAX_CL_FREQUENCY, 7000)
    elif channel == 3:
        # rotational positioner    
        ctl.SetProperty_i32(
                    d_handle, channel, ctl.Property.MAX_CL_FREQUENCY, 14000)
    # set velocity to be max allowed by the cl freq
    ctl.SetProperty_i64(d_handle, channel,
                            ctl.Property.MOVE_VELOCITY, 0)



### Calibration

In [ ]:
for channel in range(n_channels):
    # Set calibration options (start direction: forward)
    ctl.SetProperty_i32(d_handle, channel, ctl.Property.CALIBRATION_OPTIONS, 0)
    # Start calibration sequence
    ctl.Calibrate(d_handle, channel)

### Checking state

In [ ]:
state = ctl.GetProperty_i32(d_handle, 0, ctl.Property.CHANNEL_STATE)

is_moving = (state & ctl.ChannelState.ACTIVELY_MOVING) != 0

### Movement

In [ ]:
#set all channels to absolute move mode
for channel in range(n_channels):
    ctl.SetProperty_i32(d_handle, channel, ctl.Property.MOVE_MODE, ctl.MoveMode.CL_ABSOLUTE)
    ctl.SetProperty_i32(d_handle, channel, ctl.Property.HOLD_TIME, ctl.HOLD_TIME_INFINITE)
    ctl.SetProperty_i64(d_handle, channel,ctl.Property.MOVE_VELOCITY, 5*10**8)
    

In [ ]:
ctl.Move(d_handle, 0, 0, 0)

In [ ]:
ctl.Move(d_handle, 0, -200000000, 0)
time.sleep(0.5)
ctl.Move(d_handle, 0, 200000000, 0)
state = ctl.GetProperty_i32(d_handle, 0, ctl.Property.CHANNEL_STATE)
is_moving = (state & ctl.ChannelState.ACTIVELY_MOVING) != 0
print(is_moving)
time.sleep(2)
state = ctl.GetProperty_i32(d_handle, 0, ctl.Property.CHANNEL_STATE)
is_moving = (state & ctl.ChannelState.ACTIVELY_MOVING) != 0
print(is_moving)


### Setting the logical position

In [ ]:
for channel in range(3):
    ctl.SetProperty_i64(d_handle, channel, ctl.Property.POSITION, 0)

In [ ]:
ctl.Property.REFERENCING_OPTIONS.flags

In [ ]:
ctl.GetProperty_i64(d_handle, channel,ctl.Property.MOVE_VELOCITY)

In [ ]:
channel=3

In [ ]:
channel=0
ctl.SetProperty_i64(d_handle, channel,ctl.Property.MOVE_VELOCITY, 5*10**8)
ref_options = ctl.ReferencingOption.START_DIR | ctl.ReferencingOption.STOP_ON_REF_FOUND
ctl.SetProperty_i32(d_handle, channel, ctl.Property.REFERENCING_OPTIONS, ref_options)
ctl.Reference(d_handle, channel)

In [84]:
ctl.GetProperty_i64(d_handle, 1, ctl.Property.MOVE_ACCELERATION)

0

In [119]:
ctl.GetProperty_i32(d_handle, 0, ctl.Property.MAX_CL_FREQUENCY)

18500

In [ ]:
state = ctl.GetProperty_i32(
            d_handle, channel, ctl.Property.CHANNEL_STATE)
is_referenced = (state & ctl.ChannelState.IS_REFERENCED) != 0
print(is_referenced)

In [ ]:
ctl.Move(d_handle, 0, 110*10**6, 0)

In [ ]:
ctl.SetProperty_i64(d_handle, 0, ctl.Property.POSITION, 0)

In [ ]:
control.stop()

In [ ]:
ctl.Stop(d_handle, channel)

In [120]:
ctl.Close(d_handle)

### Using group commands

In [99]:
#set all channels to absolute move mode
for channel in range(n_channels):
    ctl.SetProperty_i32(d_handle, channel, ctl.Property.MOVE_MODE, ctl.MoveMode.CL_RELATIVE)
    ctl.SetProperty_i32(d_handle, channel, ctl.Property.HOLD_TIME, ctl.HOLD_TIME_INFINITE)
    ctl.SetProperty_i64(d_handle, channel,ctl.Property.MOVE_VELOCITY, 1*10**9)
    

In [106]:
def move_command():
    t_handle = ctl.OpenCommandGroup(d_handle, ctl.CmdGroupTriggerMode.DIRECT)

    ctl.Move(d_handle, 0, 0, t_handle)
    ctl.Move(d_handle, 1, 0, t_handle)

    ctl.CloseCommandGroup(d_handle, t_handle)
    t0 = time.time()
    count_movement_finished = 0
    while True:
        event = ctl.WaitForEvent(d_handle, 10000)
        print(ctl.GetEventInfo(event))
        if event.type == ctl.EventType.MOVEMENT_FINISHED:
            count_movement_finished += 1
            if count_movement_finished == 2:
                print('Group finished')
                break

In [12]:
r_id = [0]*6
t_handle = ctl.OpenCommandGroup(d_handle, ctl.CmdGroupTriggerMode.DIRECT)

ctl.Move(d_handle, 0, 10**6, t_handle)
ctl.Move(d_handle, 1, -10**6, t_handle)

ctl.CloseCommandGroup(d_handle, t_handle)

In [7]:
ctl.EventType.CMD_GROUP_TRIGGERED

<EventType.CMD_GROUP_TRIGGERED: 32784>

In [11]:
ctl.EventType.MOVEMENT_FINISHED

<EventType.MOVEMENT_FINISHED: 1>

In [107]:
t0 = time.time()
# for i in range(1):
move_command()
print(time.time() - t0)

command group triggered, handle: 13 (no error)
movement finished, channel: 0 (no error)
movement finished, channel: 1 (no error)
Group finished
0.0029680728912353516


In [39]:
ctl.EventType.MOVEMENT_FINISHED

<EventType.MOVEMENT_FINISHED: 1>

In [26]:
0.0438838005065918*256**2/60

47.9328125

In [112]:
256*256

65536

In [118]:
300/100000*256*256/60

3.2768

In [ ]:
ctl.Get

In [114]:
0.1*256

25.6

In [111]:
0.001*256**2 /60

1.0922666666666667

In [ ]:
define_stream()

stage.start_stream()
start_time = woll.get_time()
wait_for_finish()
stop_time = woll.get_time()
woll.get_data(start_time=start_time, end_time=end_time)


### to test:
- move across one of my structures
- move back across the structure
- see what is the difference

# Using the custom smaract controller 

In [1]:
import os, sys
import smaract.ctl as ctl
import numpy as np
sys.path.append('..')
from control.controllers.smaract_control import SmaractControl
import control

In [2]:

locator = ctl.FindDevices()
sc = SmaractControl(locator)

In [3]:
sc.is_referenced(3)

True

In [ ]:
sc.reference(2, direction=0)

In [ ]:
for channel in range(3):
    sc.set_position(channel, 0)

In [ ]:
sc.stop()

In [ ]:
sc.is_moving()

In [4]:
sc.get_position(0)

-584784764

In [ ]:
sc.ref

In [ ]:
sc.set_position(2, 0)

In [ ]:
sc.stop()

In [ ]:
for channel in range(sc.n_channels):
    sc.set_position(10000, channel)

In [ ]:
for channel in range(3):
    sc.set_velocity(channel, 10**6)

In [ ]:
sc.stop()

In [ ]:
sc.set_position(200000000, 0, wait=True)

In [ ]:
ctl.Close(sc.d_handle)

# Using the smaract instrument

In [ ]:
import os, sys
import smaract.ctl as ctl
import numpy as np
sys.path.append('..')
from control.controllers.smaract_control import SmaractControl
from control.instruments.basic.smaract import Smaract

locator = ctl.FindDevices()
control = SmaractControl(locator)
sci = Smaract(control, axes={"x":0, "y":1, "z":2, "phi":3})

sci.set_position({"y":0})

In [ ]:

sci.set_velocity(1000000)

In [ ]:

sci.set_position({"x":10000000})

In [ ]:
control.reference(0, direction=-1)

In [ ]:
control.

In [ ]:
control.stop()

In [ ]:
sci.define_position([0, 0, ])

In [ ]:
sci.get_data()

In [ ]:
isinstance(np.array([1, 2]), np.ndarray)

In [ ]:
sci.define_position([10000, 20000, 30000, 0])

In [ ]:
sci.is_moving()

In [ ]:
sci.define_position(1)

# Using MOKE

In [ ]:
import os, sys
sys.path.append('..')
from control.instruments.moke import Moke

In [ ]:
mk = Moke()

In [ ]:
mk.instruments['stage'].set_position({'x':0})

In [ ]:
import numpy as np
movement_range=[-1000, 1000]
step=100
np.arange(movement_range[0], movement_range[1]+step, step)

In [ ]:
import numpy as np
position = np.array([100, 0, 0, -45])
angle = -45


c, s = np.cos(np.deg2rad(angle)), np.sin(np.deg2rad(angle))
R = np.array([[c, 0, -s], [0, 1, 0], [s, 0, c]])
position.dot(R.T)

In [ ]:
R